In [5]:
import requests
from bs4 import BeautifulSoup
import re
import os

def fetch_nse_index_csv(index_url, save_dir="downloads"):
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.nseindia.com"
    }

    # Step 1: Get HTML content
    session = requests.Session()
    session.get("https://www.nseindia.com", headers=headers)  # Needed to set cookies
    response = session.get(index_url, headers=headers)

    if response.status_code != 200:
        print("Failed to fetch page")
        return

    soup = BeautifulSoup(response.content, "html.parser")

    # Step 2: Find the .csv link
    csv_link = None
    for a_tag in soup.find_all("a", href=True):
        href = a_tag["href"]
        if ".csv" in href:
            csv_link = href
            break

    if not csv_link:
        print("CSV link not found.")
        return

    # Full link (relative to base domain if not absolute)
    if not csv_link.startswith("http"):
        csv_link = "https://www.niftyindices.com" + csv_link

    print(f"Downloading CSV from: {csv_link}")

    # Step 3: Download and save the CSV
    csv_response = session.get(csv_link, headers=headers)
    if csv_response.status_code == 200:
        os.makedirs(save_dir, exist_ok=True)
        file_name = csv_link.split("/")[-1]
        file_path = os.path.join(save_dir, file_name)

        with open(file_path, "wb") as f:
            f.write(csv_response.content)

        print(f"✅ CSV saved as: {file_path}")
    else:
        print("Failed to download CSV.")

# Example usage:
fetch_nse_index_csv("https://www.nseindia.com/products-services/indices-niftymidcap100-index")

✅ CSV saved as: downloads/ind_niftymidcap100list.csv


In [18]:
import requests
from bs4 import BeautifulSoup
import os

def download_csv_from_webpage(page_url, download_folder="INDEXES"):
    try:
        # Step 1: Create the folder if it doesn't exist
        os.makedirs(download_folder, exist_ok=True)

        # Step 2: Fetch the webpage
        response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()

        # Step 3: Parse the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Step 4: Find the CSV link
        csv_link = None
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            if href.endswith('.csv'):
                csv_link = href
                break

        if not csv_link:
            print("❌ No CSV link found on the page.")
            return

        # Step 5: Handle relative link if needed
        if csv_link.startswith('/'):
            csv_link = "https://www.niftyindices.com" + csv_link

        # Step 6: Download the CSV
        csv_response = requests.get(csv_link, headers={"User-Agent": "Mozilla/5.0"})
        csv_response.raise_for_status()

        file_name = os.path.join(download_folder, os.path.basename(csv_link))

        with open(file_name, 'wb') as f:
            f.write(csv_response.content)

        print(f"✅ CSV downloaded successfully and saved at: {file_name}")

    except Exception as e:
        print(f"❌ Error: {e}")



In [19]:
download_csv_from_webpage("https://www.niftyindices.com/indices/equity/sectoral-indices/nifty-auto")

❌ Error: Invalid URL '../../../IndexConstituent/ind_niftyautolist.csv': No scheme supplied. Perhaps you meant https://../../../IndexConstituent/ind_niftyautolist.csv?
